In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

!gcloud projects list

from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client(project='clinical-entity-extraction')

# List all tables in the dataset.
dataset_ref = client.get_dataset('physionet-data.mimiciii_clinical')
tables = list(client.list_tables(dataset_ref))

print("Tables:")
for table in tables:
    table_ref = dataset_ref.table(table.table_id)
    table = client.get_table(table_ref)
    print(f"  {table.table_id}: {table.num_rows} rows")
    for schema_field in table.schema:
        print(f"  - {schema_field.name} ({schema_field.field_type})")

Authenticated
PROJECT_ID                  NAME                        PROJECT_NUMBER
clinical-entity-extraction  clinical-entity-extraction  321960627270
Tables:
  admissions: 58976 rows
  - ROW_ID (INTEGER)
  - SUBJECT_ID (INTEGER)
  - HADM_ID (INTEGER)
  - ADMITTIME (DATETIME)
  - DISCHTIME (DATETIME)
  - DEATHTIME (DATETIME)
  - ADMISSION_TYPE (STRING)
  - ADMISSION_LOCATION (STRING)
  - DISCHARGE_LOCATION (STRING)
  - INSURANCE (STRING)
  - LANGUAGE (STRING)
  - RELIGION (STRING)
  - MARITAL_STATUS (STRING)
  - ETHNICITY (STRING)
  - EDREGTIME (DATETIME)
  - EDOUTTIME (DATETIME)
  - DIAGNOSIS (STRING)
  - HOSPITAL_EXPIRE_FLAG (INTEGER)
  - HAS_CHARTEVENTS_DATA (INTEGER)
  callout: 34499 rows
  - ROW_ID (INTEGER)
  - SUBJECT_ID (INTEGER)
  - HADM_ID (INTEGER)
  - SUBMIT_WARDID (INTEGER)
  - SUBMIT_CAREUNIT (STRING)
  - CURR_WARDID (INTEGER)
  - CURR_CAREUNIT (STRING)
  - CALLOUT_WARDID (INTEGER)
  - CALLOUT_SERVICE (STRING)
  - REQUEST_TELE (INTEGER)
  - REQUEST_RESP (INTEGER)
  - R

In [ ]:
from google.colab import auth
auth.authenticate_user()
client = bigquery.Client(project='clinical-entity-extraction')

NameError: name 'bigquery' is not defined

In [ ]:
# 1. Number of Cardiac Vs Trauma Patients Admitted Each Year
query = """
    SELECT
        EXTRACT(YEAR FROM ADMITTIME) AS admit_year,
        COUNT(CASE WHEN LOWER(DIAGNOSIS) LIKE '%cardiac%' THEN 1 END) AS num_cardiac_patients,
        COUNT(CASE WHEN LOWER(DIAGNOSIS) LIKE '%trauma%' THEN 1 END) AS num_trauma_patients
    FROM `physionet-data.mimiciii_clinical.admissions`
    GROUP BY admit_year
    ORDER BY admit_year;
"""


# Run the query
query_job = client.query(query)
df = query_job.to_dataframe()
df


,admit_year,num_cardiac_patients,num_trauma_patients
0,2100,2,1
1,2101,6,7
2,2102,6,2
3,2103,6,9
4,2104,11,5
...,...,...,...
106,2206,0,0
107,2207,0,0
108,2208,0,0
109,2209,0,0


In [ ]:
# 2. Top 5 Most Common Diagnoses for Cardiac Patients
query = """
    SELECT
        d.LONG_TITLE AS diagnosis,
        COUNT(*) AS num_cases
    FROM `physionet-data.mimiciii_clinical.diagnoses_icd` d_icd
    JOIN `physionet-data.mimiciii_clinical.d_icd_diagnoses` d
        ON d_icd.ICD9_CODE = d.ICD9_CODE
    WHERE LOWER(d.LONG_TITLE) LIKE '%cardiac%'
    GROUP BY diagnosis
    ORDER BY num_cases DESC
    LIMIT 5;
"""

query_job = client.query(query)
df = query_job.to_dataframe()
df


,diagnosis,num_cases
0,Other specified cardiac dysrhythmias,2453
1,"Cardiac complications, not elsewhere classified",2343
2,Cardiac pacemaker in situ,1390
3,Cardiac arrest,1361
4,Automatic implantable cardiac defibrillator in...,733


In [ ]:
# 3. Average Length of Stay for Cardiac vs. Trauma Patients
query = """
    SELECT
        CASE
            WHEN LOWER(a.DIAGNOSIS) LIKE '%cardiac%' THEN 'Cardiac'
            WHEN LOWER(a.DIAGNOSIS) LIKE '%trauma%' THEN 'Trauma'
            ELSE 'Other'
        END AS patient_type,
        ROUND(AVG(i.LOS), 2) AS avg_length_of_stay
    FROM `physionet-data.mimiciii_clinical.admissions` a
    JOIN `physionet-data.mimiciii_clinical.icustays` i
        ON a.HADM_ID = i.HADM_ID
    WHERE LOWER(a.DIAGNOSIS) LIKE '%cardiac%' OR LOWER(a.DIAGNOSIS) LIKE '%trauma%'
    GROUP BY patient_type;
"""

query_job = client.query(query)
df = query_job.to_dataframe()
df


,patient_type,avg_length_of_stay
0,Trauma,4.92
1,Cardiac,3.95


In [ ]:
# 4. Readmission Rate of Cardiac Patients Within 30 Days
query = """
    SELECT
        COUNT(*) AS num_readmissions,
        (COUNT(*) * 100.0) / (SELECT COUNT(*) FROM `physionet-data.mimiciii_clinical.admissions` WHERE LOWER(DIAGNOSIS) LIKE '%cardiac%') AS readmission_rate
    FROM (
        SELECT a1.SUBJECT_ID, a1.HADM_ID
        FROM `physionet-data.mimiciii_clinical.admissions` a1
        JOIN `physionet-data.mimiciii_clinical.admissions` a2
            ON a1.SUBJECT_ID = a2.SUBJECT_ID
            AND a1.HADM_ID <> a2.HADM_ID
            AND a2.ADMITTIME BETWEEN a1.DISCHTIME AND TIMESTAMP_ADD(a1.DISCHTIME, INTERVAL 30 DAY)
        WHERE LOWER(a1.DIAGNOSIS) LIKE '%cardiac%'
    ) readmitted_patients;
"""

query_job = client.query(query)
df = query_job.to_dataframe()
df


,num_readmissions,readmission_rate
0,35,4.137116


In [ ]:
# 5. Mortality Rate of Cardiac Patients in ICU
query = """
    SELECT
        i.FIRST_CAREUNIT,
        COUNT(*) AS num_patients,
        SUM(a.HOSPITAL_EXPIRE_FLAG) AS num_deaths,
        ROUND(SUM(a.HOSPITAL_EXPIRE_FLAG) * 100.0 / COUNT(*), 2) AS mortality_rate
    FROM `physionet-data.mimiciii_clinical.admissions` a
    JOIN `physionet-data.mimiciii_clinical.icustays` i
        ON a.HADM_ID = i.HADM_ID
    WHERE LOWER(a.DIAGNOSIS) LIKE '%cardiac%'
    GROUP BY i.FIRST_CAREUNIT
    ORDER BY mortality_rate DESC;
"""

query_job = client.query(query)
df = query_job.to_dataframe()
df


,FIRST_CAREUNIT,num_patients,num_deaths,mortality_rate
0,MICU,94,46,48.94
1,TSICU,9,4,44.44
2,SICU,19,5,26.32
3,CCU,496,103,20.77
4,CSRU,260,10,3.85


In [ ]:
# 6. Top 5 Prescribed Medications for Cardiac Patients
query = """
    SELECT
        p.DRUG_NAME_GENERIC,
        COUNT(*) AS num_prescriptions
    FROM `physionet-data.mimiciii_clinical.prescriptions` p
    JOIN `physionet-data.mimiciii_clinical.admissions` a
        ON p.HADM_ID = a.HADM_ID
    WHERE LOWER(a.DIAGNOSIS) LIKE '%cardiac%'
    GROUP BY p.DRUG_NAME_GENERIC
    ORDER BY num_prescriptions DESC
    LIMIT 5;
"""

query_job = client.query(query)
df = query_job.to_dataframe()
df


,DRUG_NAME_GENERIC,num_prescriptions
0,None,25988
1,Furosemide,2580
2,Potassium Chloride,1980
3,Metoprolol Tartrate,1805
4,Sodium Chloride 0.9% Flush,1552


In [ ]:
# 7. Common Lab Tests Ordered for Trauma Patients
query = """
    SELECT
        d.LABEL AS lab_test,
        COUNT(*) AS num_orders
    FROM `physionet-data.mimiciii_clinical.labevents` l
    JOIN `physionet-data.mimiciii_clinical.d_labitems` d
        ON l.ITEMID = d.ITEMID
    JOIN `physionet-data.mimiciii_clinical.admissions` a
        ON l.HADM_ID = a.HADM_ID
    WHERE LOWER(a.DIAGNOSIS) LIKE '%trauma%'
    GROUP BY d.LABEL
    ORDER BY num_orders DESC
    LIMIT 5;
"""

query_job = client.query(query)
df = query_job.to_dataframe()
df


,lab_test,num_orders
0,Glucose,9320
1,pH,8267
2,Hematocrit,7786
3,Hemoglobin,7094
4,pO2,6809


In [ ]:
# 8. ICU Admission Rate for Cardiac vs. Trauma Patients
query = """
    SELECT
        CASE
            WHEN LOWER(a.DIAGNOSIS) LIKE '%cardiac%' THEN 'Cardiac'
            WHEN LOWER(a.DIAGNOSIS) LIKE '%trauma%' THEN 'Trauma'
        END AS patient_type,
        COUNT(DISTINCT a.HADM_ID) AS total_admissions,
        COUNT(DISTINCT i.ICUSTAY_ID) AS icu_admissions,
        ROUND(COUNT(DISTINCT i.ICUSTAY_ID) * 100.0 / COUNT(DISTINCT a.HADM_ID), 2) AS icu_admission_rate
    FROM `physionet-data.mimiciii_clinical.admissions` a
    LEFT JOIN `physionet-data.mimiciii_clinical.icustays` i
        ON a.HADM_ID = i.HADM_ID
    WHERE LOWER(a.DIAGNOSIS) LIKE '%cardiac%' OR LOWER(a.DIAGNOSIS) LIKE '%trauma%'
    GROUP BY patient_type;
"""

query_job = client.query(query)
df = query_job.to_dataframe()
df


,patient_type,total_admissions,icu_admissions,icu_admission_rate
0,Trauma,601,626,104.16
1,Cardiac,846,878,103.78


In [ ]:
# 9. Insurance Type Distribution Among Cardiac Patients
query = """
    SELECT
        INSURANCE,
        COUNT(*) AS num_patients,
        ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM `physionet-data.mimiciii_clinical.admissions` WHERE LOWER(DIAGNOSIS) LIKE '%cardiac%'), 2) AS percentage
    FROM `physionet-data.mimiciii_clinical.admissions`
    WHERE LOWER(DIAGNOSIS) LIKE '%cardiac%'
    GROUP BY INSURANCE
    ORDER BY num_patients DESC;
"""

query_job = client.query(query)
df = query_job.to_dataframe()
df


,INSURANCE,num_patients,percentage
0,Medicare,494,58.39
1,Private,279,32.98
2,Medicaid,50,5.91
3,Government,15,1.77
4,Self Pay,8,0.95


In [ ]:
# 10. Age Distribution of Trauma Patients
query = """
    SELECT
        CASE
            WHEN TIMESTAMP_DIFF(a.ADMITTIME, p.DOB, YEAR) < 18 THEN '0-17'
            WHEN TIMESTAMP_DIFF(a.ADMITTIME, p.DOB, YEAR) BETWEEN 18 AND 40 THEN '18-40'
            WHEN TIMESTAMP_DIFF(a.ADMITTIME, p.DOB, YEAR) BETWEEN 41 AND 65 THEN '41-65'
            ELSE '65+'
        END AS age_group,
        COUNT(*) AS num_patients
    FROM `physionet-data.mimiciii_clinical.admissions` a
    JOIN `physionet-data.mimiciii_clinical.patients` p
        ON a.SUBJECT_ID = p.SUBJECT_ID
    WHERE LOWER(a.DIAGNOSIS) LIKE '%trauma%'
    GROUP BY age_group
    ORDER BY num_patients DESC;
"""

query_job = client.query(query)
df = query_job.to_dataframe()
df


,age_group,num_patients
0,18-40,239
1,41-65,216
2,65+,133
3,0-17,13
